In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.2 MB/s 
     |████████████████████████████████| 7.6 MB 34.3 MB/s 
     |████████████████████████████████| 163 kB 64.4 MB/s 


In [ ]:
import warnings
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import datetime
import random
import json

import torch
import torch.optim as optim
from transformers import BertTokenizer,  BertForMaskedLM ,AdamW ,BertConfig ,get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset , random_split
from torch.utils.data import DataLoader ,RandomSampler, SequentialSampler

from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
%cd "/content/drive/MyDrive/AI5 MLOPS/"

/content/drive/.shortcut-targets-by-id/1IBzU3wncSMdMkz8e2pEnfntllTMVQ3O0/AI5 MLOPS


In [ ]:
data  = pd.read_csv("papers_with_abstract.csv")
print("Data Shape :" , data.shape)

Data Shape : (9313, 6)


In [ ]:
data = data.drop(["Unnamed: 0","source_id"],axis = 1)
data['abstract'] = data['abstract'].replace(r'\n', ' ', regex=True) 
data['full_text'] = data['full_text'].replace(r'\n', ' ', regex=True)

Toenizer and model Initialization 

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Get the SciBERT pretrained model path from Allen AI repo
pretrained_model = 'allenai/scibert_scivocab_uncased'

# Get the tokenizer from the previous path
sciBERT_tokenizer = BertTokenizer.from_pretrained(pretrained_model, 
                                          do_lower_case=True)

#objective of the masked language model is to predict the masked token, the label and the inputs are the same                                        do_lower_case=True)
model = BertForMaskedLM.from_pretrained(pretrained_model)

Downloading:   0%|          | 0.00/228k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 202 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (31090, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

Preparing dataset

In [ ]:
!pwd

/content/drive/.shortcut-targets-by-id/1IBzU3wncSMdMkz8e2pEnfntllTMVQ3O0/AI5 MLOPS


In [ ]:
input_ids = []
attention_masks = []
abstract = data['abstract'].tolist()

# For every sentence...
for sent in abstract:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = sciBERT_tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 150,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
dataset = TensorDataset(input_ids, attention_masks)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
#the authors of the original BERT paper recommend learning rates of 3e-4, 1e-4, 5e-5, and 3e-5 as good starting points
lr = 5e-5

#AdamW is a variant of the optimizer Adam that has an improved implementation of weight decay.
optimizer = optim.AdamW(model.parameters(), lr=lr)
max_len = 150
epochs = 20

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

seed_val = 66

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
model.to(device)
model.train()

In [ ]:
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []
val_check =[]

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    running_loss = 0

     # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

      
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)

        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch)
        # It returns different numbers of parameters depending on what arguments
        # are given and what flags are set. For our usage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        output = model(b_input_ids,
                        token_type_ids=None, 
                        attention_mask=b_input_mask,
                        labels=b_input_ids)
        loss = output.loss

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        #print(loss)

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

        running_loss += loss.item() * b_input_ids.size(0)
        print(running_loss)
    # Calculate the average loss over all of the batches.
    avg_train_loss = running_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
            
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            output = model(b_input_ids, 
                                  token_type_ids=None, 
                                  attention_mask=b_input_mask,
                                  labels=b_input_ids)
            loss = output.loss
            logits = output.logits
            
        # Accumulate the validation loss.
        running_loss += loss.item() * b_input_ids.size(0)

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))



    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avgtrain_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
  
    )
    val_check.append(val_loss)
    checkpoint = {
              "epoch": epoch_i,
              "model_state": model.state_dict(),
              "optim_state": optimizer.state_dict()
              }
    FILE1 = "checkpoint.pth"
    FILE2 = "checkpoint_buf.pth"
    if (val_check[-1] > val_check[-2]):
      FILE1 = "checkpoint.pth"
      torch.save(checkpoint, FILE)
      if (epovh_i>2 ) and (val_check[-2] < val_check[-3]):
          FILE = "checkpoint_buf.pth"
          torch.save(checkpoint, FILE)
      else (epovh_i>3 ) and((val_check[-2] > val_check[-3]) and (val_check[-3] > val_check[-4])):
          break


print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
model.save_pretrained("Fine_Tuned_BertForMaskedLM")

with open("training_stats.json", "w") as outfile:
    json.dump(training_stats, outfile)        

In [ ]:
val_check = [0.1,0.01,0.001,0.003,0.01,0.1]
for epoch_i in range(0, 15):
      print(epoch_i)

      print("save")

      if (epoch_i>2 ):
        if (val_check[-1] > val_check[-2]):
          if  (val_check[-2] < val_check[-3]):
              print("save_buf")

      if (epoch_i >3 ) :
        print(val_check[-1],val_check[-2],val_check[-3])

        if (val_check[-1] > val_check[-2]):
          if ((val_check[-2] > val_check[-3]) & (val_check[-3] < val_check[-4]) ) : 
              print("save_buf")
          else :
              break
        if (val_check[-1] > val_check[-2])

0
save0
save
1
save
2
save
3
save
4
save
0.1 0.01 0.003


In [ ]:
# Display floats with two decimal places.
pd.set_option('precision', 2)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

In [ ]:
import matplotlib.pyplot as plt
% matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# Label the plot.
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4])

plt.show()

In [ ]:
# since = time.time()

# # best_model_wts = copy.deepcopy(model.state_dict())
# # best_acc = 0.0

# for epoch in range(epochs):
#   print('Epoch {}/{}'.format(epoch, epochs - 1))
#   print('-' * 10)
#   epoch_losses = []
#   for j, batch in enumerate(pd.read_csv("/content/drive/MyDrive/AI5 MLOPS/papers_with_abstract.csv", chunksize=40)):
#       batch['abstract'] = batch['abstract'].replace(r'\n', ' ', regex=True) 
#       abs = batch['abstract'].tolist()
  
#       encoded_dict = sciBERT_tokenizer.batch_encode_plus(
#                           abs,                      # Sentence to encode.
#                           add_special_tokens = True, # Add '[CLS]' and '[SEP]'
#                           max_length = max_len,           # Pad & truncate all sentences.
#                           pad_to_max_length = True,
#                           truncation=True,
#                           return_attention_mask = True,   # Construct attn. masks.
#                           return_tensors = 'pt',     # Return pytorch tensors.
#                       )
#       input_ids = encoded_dict['input_ids'].to(device)
#       print(input_ids)
      
#       o = model(input_ids, labels=input_ids )
#       # print(len(o),"next",o.loss,o.logits)
#       loss = o.loss
#       loss_score = loss.item()
#       print(loss_score)

#       # if phase == 'train':
#       loss.backward()
#       torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
#       optimizer.step()
#       optimizer.zero_grad()

#       # if phase == 'val' and epoch_acc > best_acc:
#       #           best_acc = epoch_acc
#       #           best_model_wts = copy.deepcopy(model.state_dict())
# # model.save_pretrained(path + "Fine_Tuned_BertForMaskedLM")